In [1]:
from sklearn import datasets
import numpy as np
# import kmeans
from sklearn.cluster import KMeans

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [14]:
sse_values = []
k_values = range(2, 10)  # Choose a range of values for the number of clusters
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
    mean = np.mean(kmeans, axis=0)
    squared_diff = (kmeans - mean) ** 2
    sse = np.sum(squared_diff)
    sse_values.append(sse)

# Plot the SSE values against the number of clusters to analyze the results
import matplotlib.pyplot as plt
plt.plot(k_values, sse_values, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')
plt.title('Elbow Method to Find Optimal Number of Clusters')
plt.show()

c:\Users\dimit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\dimit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


AxisError: axis 0 is out of bounds for array of dimension 0

In [17]:

def bkmeans(x_data, num_of_clusters , iterations):
    indices = []
    sse_values = []
    clusters = []
    clusters.append(x_data)
    current_X_data = x_data
    for i in range(1, iterations+1):
        kmeans = KMeans(n_clusters=2, random_state=0).fit(current_X_data)
        cluster1 = current_X_data[kmeans.labels_ == 0]
        cluster2 = current_X_data[kmeans.labels_ == 1]
        sse1 = np.sum((cluster1 - np.mean(cluster1))**2)
        sse2 = np.sum((cluster2 - np.mean(cluster2))**2)
        if sse1 > sse2:
            current_X_data = cluster1
            clusters.append(cluster2)
            sse_values.append(sse1)
        else:
            current_X_data = cluster2
            clusters.append(cluster1)
            sse_values.append(sse2)
        indices.append(kmeans.labels_)
        if len(sse_values) > 1:
            sse_diff = sse_values[-2] - sse_values[-1]
            if sse_diff < 200:
                break
    return indices, sse_values


def bkmeans_new(X_data, num_of_clusters, iterations):
    indices = np.emp
    clusters = []
    clusters.append(X_data)
    current_X_data = X_data
    current_sse = 0
    for i in range(1, num_of_clusters):
        for i in range(iterations):
            kmeans = KMeans(n_clusters=2, random_state=0).fit(current_X_data)
            current_sse = kmeans.inertia_






indices, sse_values = bkmeans(X, 5, 10)
    
   



c:\Users\dimit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\dimit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\dimit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\dimit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environme